In [1]:
import csv
import zipfile
import pandas as pd # pandas has one too
from pathlib import Path
import awswrangler as wr
import boto3

In [ ]:
def save_compressed_df(df, dirPath, fileName):
    """Save a Pandas dataframe as a zipped .csv file.

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        Input dataframe.
    dirPath : str or pathlib.PosixPath
        Parent directory of the zipped file.
    fileName : str
        File name without extension.
    """

    dirPath = Path(dirPath)
    path_zip = dirPath / f'{fileName}.csv.zip'
    txt = df.to_csv(index=False)
    with zipfile.ZipFile(path_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.writestr(f'{fileName}.csv', txt)

In [2]:
df = pd.read_csv('nba_tweets.csv')

/home/jacob/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
df.to_csv('nba_tweets2.csv', index = None, sep = ",", header=True)

In [33]:
# nba_tweets.csv is 11.7 mb, zips are 3.7 mbs
df.to_csv('nba_tweets.zip', compression = 'zip', index = None, sep = ",", header=True, encoding='utf-8-sig')   # zip archive is collection of compressed files.  every file is uniquely compressed.
df.to_csv('nba_tweets.gz', compression = 'gzip', index = None, sep = ",", header=True, encoding='utf-8-sig')   
df.to_csv('nba_tweets.tar.gz', compression = 'gzip', index = None, sep = ",", header=True, encoding='utf-8-sig') 

# zip archive is collection of compressed files.  every file is uniquely compressed.
# compressed collection of uncompressed files.  must decompress everything to access files within
# # tar bundles everything up, gz compresses that tarball

In [34]:
df_zip = pd.read_csv('nba_tweets.zip', compression = 'zip', error_bad_lines=False)
df_gzip = pd.read_csv('nba_tweets.gz', compression = 'gzip', error_bad_lines=False)
df_tar = pd.read_csv('nba_tweets.tar.gz', compression = 'gzip', error_bad_lines=False)

/home/jacob/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/home/jacob/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
s3 = boto3.resource('s3')

In [38]:
wr.s3.to_parquet(
    df = df,
    path = "s3://jacobsbucket97/sample_files/nba_tweets.parquet"
)

{'paths': ['s3://jacobsbucket97/sample_files/nba_tweets.parquet'],
 'partitions_values': {}}

In [ ]:
wr.s3.to_parquet(
    df = df,
    path = "s3://jacobsbucket97/sample_files/nba_tweets.parquet"
)

In [5]:
wr.s3.to_parquet(
    df = df,
    path = "s3://jacobsbucket97/sample_files/nba_tweets_snappy2.snappy",
    compression = "snappy"
)

# wr.s3.to_parquet(
#     df = df,
#     path = "s3://jacobsbucket97/sample_files/nba_tweets_zip.gzip.parquet",
#     compression = 'gzip'
# )

{'paths': ['s3://jacobsbucket97/sample_files/nba_tweets_snappy2.snappy'],
 'partitions_values': {}}

In [8]:
# df2 = wr.s3.read_parquet(
#     path = "s3://jacobsbucket97/sample_files/nba_tweets_snappy2.snappy"
# )

df3 = wr.s3.read_parquet(
    path = "s3://jacobsbucket97/sample_files/nba_tweets_zip.gzip.parquet"
)

In [9]:
wr.s3.to_parquet(
    df = df,
    path = "s3://jacobsbucket97/sample_files/nba_tweets_zip2.parquet",
    compression = 'gzip'
)
# 2.1 mb for compressed parquet, 3.0 mb for normal parquet, 11.2 for csv, 3.5mb for zipped csv.
# i believe the idea is compressed parquet might take longer to read from, but its the most efficient storage option.

{'paths': ['s3://jacobsbucket97/sample_files/nba_tweets_zip2.parquet'],
 'partitions_values': {}}

In [5]:
# read in first 1000 rows
df = pd.read_csv('nba_tweets.csv', nrows = 10) #skiprows = 998